In [6]:
from typing import *

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

import os

from lib.dnn.utils import KerasTrainer
from core.utils.cached_optimizer import CachedOptimizer
from lib.dnn.layers import Delta, MovingAverage, Percentage


In [2]:
class Trial0Trainer(KerasTrainer):
	def __init__(self, seq_len, hidden_layers: List[int], *args,
	             loss="binary_crossentropy",
	             optimizer="adam",
	             hidden_activation="relu",
	             delta=True,
	             average_window=0,
	             percentage=True,
	             **kwargs):
		super(Trial0Trainer, self).__init__(*args, **kwargs)
		self.__seq_len = seq_len
		self.__hidden_layers = hidden_layers
		self.__loss = loss
		self.__optimizer = optimizer
		self.__hidden_activation = hidden_activation
		self.__delta = delta
		self.__percentage = percentage
		self.__average_window = average_window

	def __get_currency_pairs(self, df: pd.DataFrame) -> List[Tuple[str, str]]:
		DELIMITER = "/"
		return [(pair.split(DELIMITER)[0], pair.split(DELIMITER)[1]) for pair in
		        set(df["base_currency"] + DELIMITER + df["quote_currency"])]  # TODO FIND A CLEAR WAY

	def __prepare_for_pair(self, sequence: np.ndarray, seq_len: int):
		data_len = sequence.shape[0] - seq_len
		X = np.zeros((data_len, seq_len))
		y = np.zeros((data_len,))
		for i in range(data_len):
			X[i] = sequence[i:i + seq_len]
			if sequence[i] > sequence[i - 1]:
				y[i] = 1
			else:
				y[i] = 0
		return X, y

	def moving_average(self, sequence, window_size):
		data_len = sequence - window_size + 1
		final_sequence = np.zeros(data_len)
		
		for i in range(data_len):
			final_sequence[i] = np.mean(sequence[i: i+window_size])

		return final_sequence

	def _prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
		currency_pairs = self.__get_currency_pairs(data)

		X = []
		y = []

		for base_currency, quote_currency in currency_pairs:
			pair_sequence = data[data["base_currency"] == base_currency][
				data[data["base_currency"] == base_currency]["quote_currency"] == quote_currency][
				"c"].to_numpy()  # TODO
			if self.__average_window != 0:
				pair_sequence = self.moving_average(pair_sequence, self.__average_window)
			pair_X, pair_y = self.__prepare_for_pair(pair_sequence, self.__seq_len)
			X += list(pair_X)
			y += list(pair_y)

		return np.array(X), np.array(y)

	def _create_model(self) -> keras.Model:
		input_layer = keras.layers.Input(shape=self.__seq_len)

		pre_hidden_layer = input_layer

		if self.__average_window != 0:
			pre_hidden_layer = MovingAverage(self.__average_window)(pre_hidden_layer)

		pre_hidden_layer_copy = pre_hidden_layer[:, : -1]

		if self.__delta:
			pre_hidden_layer = Delta()(pre_hidden_layer)
		else:
			pre_hidden_layer = pre_hidden_layer[:, 1:]

		if self.__percentage:
			pre_hidden_layer = tf.concat(
				(
					tf.reshape(pre_hidden_layer, (-1, 1, pre_hidden_layer.shape[1])),
					tf.reshape(pre_hidden_layer_copy, (-1, 1, pre_hidden_layer.shape[1]))
				),
				axis=1
			)
			pre_hidden_layer = Percentage()(pre_hidden_layer)

		output_layer = keras.layers.Dense(1, activation="sigmoid")

		if len(self.__hidden_layers) == 0:
			output_layer = output_layer(pre_hidden_layer)
		else:
			hidden_layer = keras.layers.Dense(self.__hidden_layers[0], activation=self.__hidden_activation)(
				pre_hidden_layer)
			for layer_size in self.__hidden_layers[1:]:
				hidden_layer = keras.layers.Dense(layer_size, activation=self.__hidden_activation)(hidden_layer)
			output_layer = output_layer(hidden_layer)

		model = keras.Model(inputs=input_layer, outputs=output_layer)

		return model

	def _compile_model(self, model: keras.Model):
		model.compile(loss=self.__loss, optimizer=self.__optimizer, metrics=["accuracy"])


In [3]:
class Trial0Optimizer(CachedOptimizer):

    def __init__(self, user):
        self.__samples = None
        super(Trial0Optimizer, self).__init__(user)
        
    def __load_samples(self):
        return eval(open("temp/notebooks/samples.txt").read())
    
    def _get_samples(self) -> List[Dict]:
        if self.__samples is None:
            self.__samples = self.__load_samples()
        return self.__samples

    # def __generate_possible_layers(self, layer_sizes, depth):
    #     if depth == 1:
    #         return [[size] for size in layer_sizes]
	# 
    #     layers = []
    #     returned_layers = self.__generate_possible_layers(layer_sizes, depth - 1)
    #     layers += returned_layers
    #     for size in layer_sizes:
    #         for layer in returned_layers:
    #             layers += [[size] + layer]
	# 
    #     return layers
	# 
    # def __generate_hidden_layers(self, base, min_power, max_power, max_depth) -> List[List[int]]:
    #     print("[+]Generating Hidden Layer Sizes...")
	# 
    #     layer_sizes = [base ** i for i in range(min_power, max_power)]
	# 
    #     return self.__generate_possible_layers(layer_sizes, max_depth)
	# 
    # def _generate_param_values(self) -> Dict:
    #     return {
    #         "seq_len": [2 ** (i) for i in range(2, 6)],
    #         "hidden_layers": self.__generate_hidden_layers(self.__base, self.__min_power, self.__max_power,
    #                                                        self.__max_depth),
    #         "loss": ["binary_crossentropy"],
    #         "optimizer": ["adam"],
    #         "hidden_activation": ["relu"],
    #         "delta": [True, False],
    #         "percentage": [True, False],
    #         "average_window": [0],
    #         
    #     }

    def _create_trainer(self, params) -> KerasTrainer:
        print("[+]Creating Trainer...")
        return Trial0Trainer(**params, export_path="temp/dummyModel.h5", data_path="Data/Minutes/AUD-CAD.csv", epochs=2)


In [12]:
def generate_data(round_size, iterations):
	sequence = []
	for i in range(iterations):
		for v in range(-round_size, round_size):
			sequence.append(abs(v))
	
	df = pd.DataFrame(np.array(sequence), columns=["c"])
	df["base_currency"] = ["USD"]*len(df)
	df["quote_currency"] = ["EUR"]*len(df)
	return df
	



In [13]:
df = generate_data(1000, 5)

In [14]:
df.to_csv("temp.csv")

In [17]:
trainer = Trial0Trainer(64, [256, 1024, 64], data_path="temp.csv", export_path="model.h5", epochs=5)

In [18]:
trainer.start()

[+]Starting Training...
[+]Loading Data: temp.csv
[+]Splitting Data...
[+]Model Summary
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
delta (Delta)                   (None, 63)           0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 63)           0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.reshape_2 (TFOpLambda)       (None, 1, 63)        0           delta[0][0]                      
____

  1/218 [..............................] - ETA: 1:29 - loss: nan - accuracy: 0.1250

 11/218 [>.............................] - ETA: 1s - loss: nan - accuracy: 0.4688  

 21/218 [=>............................] - ETA: 1s - loss: nan - accuracy: 0.4732

 33/218 [===>..........................] - ETA: 0s - loss: nan - accuracy: 0.4915

...

39/94 [===========>..................] - ETA: 0s - loss: nan - accuracy: 0.5128 

74/94 [======================>.......] - ETA: 0s - loss: nan - accuracy: 0.5114

94/94 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5119


[+]Saving Model to model.h5


(<tensorflow.python.keras.callbacks.History at 0x7f31c4604880>,
 [nan, 0.511908769607544])

In [22]:
model = keras.models.load_model("model.h5", custom_objects={layer.__name__: layer for layer in [Delta, Percentage, MovingAverage]})

In [25]:
model.predict(np.arange(64).reshape((1, -1)))

array([[nan]], dtype=float32)